In [182]:
from bs4 import BeautifulSoup
import json # library to handle JSON files
import numpy as np
import pandas as pd
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import urllib
# import k-means from clustering stage
from sklearn.cluster import KMeans
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge


# First we'll use Beautiful soup to parse the wikipedia page

In [183]:
#use beautiful soup to scrape wikipedia page
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [184]:
# read the contents of the wikipedia page
response = urllib.request.urlopen(url)
rawhtml = response.read()

In [185]:
soup = BeautifulSoup(rawhtml, 'html.parser')

In [188]:
#find the table with the listing of postal codes (it has class wikitable)
# then loop through the rows in the table and pull out the values of hte table cells

pclist = []
for table_row in soup.select("table.wikitable tr"):
    cells = table_row.findAll('td')
    #row = [table_row.text.strip() for table_row in cells]
    #pclist.append(row)
    #note: do not include "Not assigned" values or grayed out values
    if len(cells) > 0:
        pc = cells[0].text.strip()
        b = cells[1].text.strip()
        n = cells[2].text.strip()

        if (b == "Not assigned" and n=="Not assigned"):
           #skip row
            a="skipping"
            #print('skipping')
       # elif (b == "Not assigned"):
       #     pclist.append([pc,n,n])
        elif(n=="Not assigned"):
            #if the neighbourhood is not assigned, then subsitute in the borough
            pclist.append([pc,b,b])
        else:
            pclist.append([pc,b,n])
            

toronto_df = pd.DataFrame(pclist, columns=["FSA", "Borough", "Neighborhood"])
toronto_df.head(15)

,FSA,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


### create a dataframe out of the retrieved info

In [189]:
#find the table with the listing of postal codes (it has class wikitable)
# then loop through the rows in the table and pull out the values of hte table cells

pclist = []
prevPC = "" # stores the previous postal code to see if we need to append neighbourhoods or create a new row
currNeighbourhoods = ""
prevBorough = ""
for table_row in soup.select("table.wikitable tr"):
    cells = table_row.findAll('td')
    if len(cells) > 0:
        pc = cells[0].text.strip()
        b = cells[1].text.strip()
        n = cells[2].text.strip()
        if ((b == "Not assigned" and n=="Not assigned") or b == "blank"):
           #skip row
            a="skipping"
        else:
            if (n=="Not assigned"):
                n = b; # set the neighbourhood equal to the borough
            
            if (pc == prevPC):
                # append neighbourhoods into comma delimited string
                currNeighbourhoods = currNeighbourhoods + ", " + n
                 
            else:
                # new postal code, so write out previous row
                if (prevPC != ""):
                    pclist.append([prevPC,prevBorough,currNeighbourhoods])
                prevPC = pc # set a new PrevPC value for next loop
                prevBorough = b
                currNeighbourhoods = n # start a new neighbourhoods string
               
            
toronto_df = pd.DataFrame(pclist, columns=["PostalCode", "Borough", "Neighborhood"])
toronto_df.head(15)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [190]:
# confirm the shape
toronto_df.shape

(102, 3)

### Now that we have our data frame, get the latitude and longitude for each postal code and add to the data frame

In [191]:
!pip install geocoder
import geocoder

Requirement not upgraded as not directly required: geocoder in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: ratelim in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: future in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: click in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: decorator in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from ratelim->geocoder)
Requirement not upgraded as not directly required: chardet<3.1.0,

In [192]:
# get geocode for postal codes
# initialize your variable to None
lat_lng_coords = None
# create lists to store our new lats and longs
lats = []
longs=[]
#loop through our dataframe and look up the lat/long of each postal code
for index, row in toronto_df.iterrows():
    #print (row["PostalCode"])
    postal_code=row["PostalCode"]
    # loop until you get the coordinates
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng

    #latitude = lat_lng_coords[0]
    #longitude = lat_lng_coords[1]
    lats.append(lat_lng_coords[0])
    longs.append(lat_lng_coords[1])
    #print ('postal code {} had latitude {} and longitude {}'.format(postal_code, latitude, longitude))

toronto_df['Latitude'] = lats
toronto_df['Longitude'] = longs
toronto_df.head() #preview our results

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


### check the size of our df

In [193]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_df['Borough'].unique()),
        toronto_df.shape[1]
    )
)

The dataframe has 11 boroughs and 5 neighborhoods.


### get the lat and long of Toronto in general for starting our map


In [194]:
address = 'Toronto, ON'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


### create a map of Toronto with labelled neighbourhoods

In [195]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)
        #parse_html=False).add_to(map_newyork)  
    
map_toronto

### filter our dataframe to only include Boroughs that mention "toronto" specifically

In [196]:
#lets focus in on only boroughs that contain the word Toronto
toronto_data = toronto_df[toronto_df['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [197]:
#now redo our map
# try setting our default lat and long to the first row in our dataframe
latitude = toronto_data.iat[1,3]
longitude = toronto_data.iat[1,4]
map_toronto2 = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto2)

    
map_toronto2

In [236]:
# The code was removed by Watson Studio for sharing.

Your credentails are hidden:


### explore the first neighbourhood in our dataset

In [199]:
#toronto_data.loc[1, 'Neighborhood']
neighbourhood = toronto_data.iat[1,2]
# we already have the lat and long from previous section
print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood, 
                                                              latitude, 
                                                               longitude))

Latitude and longitude values of Ryerson, Garden District are 43.6571618, -79.3789371.


### get the top 100 venues

In [200]:
LIMIT=100
radius = 500
# type your answer here
url='https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID,CLIENT_SECRET,VERSION,
        latitude,longitude,
        radius, 
    LIMIT)
print(url)

https://api.foursquare.com/v2/venues/explore?client_id=VQJYUJI3VOX00PNBOELISNJFSGHZLPFOEIOWP44RWX054HDB&client_secret=GW34Z0Z33ZINNGZ4R51OUYIBMAAAAPZETEYHZACESCB5VRHE&v=20180605&ll=43.6571618,-79.3789371&radius=500&limit=100


In [201]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5b928ffa9fb6b716d6be59d8'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-57eda381498ebe0e6ef40972-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/apparel_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d103951735',
         'name': 'Clothing Store',
         'pluralName': 'Clothing Stores',
         'primary': True,
         'shortName': 'Apparel'}],
       'id': '57eda381498ebe0e6ef40972',
       'location': {'address': '220 Yonge St',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'at Dundas St W',
        'distance': 195,
        'formattedAddress': ['220 Yonge St (at Dundas St W)',
         'Toronto ON M5B 2H1',
         'Canada'],
        'labeledLatLngs':

In [202]:
# let's borrow this function from the lab to extract the venue type
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### use above function to convert json data to data frame

In [203]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,UNIQLO ユニクロ,Clothing Store,43.655910,-79.380641
1,Silver Snail Comics,Comic Shop,43.657031,-79.381403
2,Burrito Boyz,Burrito Place,43.656265,-79.378343
3,Tacos 101,Taco Place,43.656636,-79.376968
4,Blaze Pizza,Pizza Place,43.656518,-79.380015


In [204]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


### the below function (from the lab) runs through each neighbourhood in Toronto and finds the nearest venues

In [205]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [206]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],latitudes=toronto_data['Latitude'],longitudes=toronto_data['Longitude'])
toronto_venues.head()

Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
The Danforth West, Riverdale
Design Exchange, Toronto Dominion Centre
Brockton, Exhibition Place, Parkdale Village
The Beaches West, India Bazaar
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North, Forest Hill West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
Harbord, University of Toronto
Runnymede, Swansea
Moore Park, Summerhill East
Chinatown, Grange Park, Kensington Market
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown, St. James Town
Fir

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront, Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront, Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront, Regent Park",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
3,"Harbourfront, Regent Park",43.65426,-79.360636,Cooper Koo YMCA,43.653191,-79.357947,Gym / Fitness Center
4,"Harbourfront, Regent Park",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


In [207]:
#check the size of our df
print(toronto_venues.shape)

(1700, 7)


In [208]:
# get count of venues per neighbourhood
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,58,58,58,58,58,58
"Brockton, Exhibition Place, Parkdale Village",22,22,22,22,22,22
Business reply mail Processing Centre969 Eastern,16,16,16,16,16,16
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",14,14,14,14,14,14
"Cabbagetown, St. James Town",46,46,46,46,46,46
Central Bay Street,85,85,85,85,85,85
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,16,16,16,16,16,16


In [209]:
#how many unique categories are there?
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 238 uniques categories.


## Now we will analyze each neighbourhood

In [210]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
manhattan_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [211]:
toronto_onehot.shape

(1700, 238)

In [212]:
#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [213]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Yoga Studio
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,...,0.000000,0.000000,0.00000,0.010000,0.000000,0.000000,0.00,0.010000,0.000000,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
3,Business reply mail Processing Centre969 Eastern,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.062500
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.000000,0.071429,0.071429,0.071429,0.142857,0.142857,0.142857,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
5,"Cabbagetown, St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021739,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
6,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011765,...,0.000000,0.000000,0.00000,0.011765,0.000000,0.000000,0.00,0.011765,0.000000,0.011765
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.010000,0.000000,0.00000,0.060000,0.000000,0.040000,0.00,0.010000,0.000000,0.000000
8,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
9,Church and Wellesley,0.012195,0.012195,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.024390,...,0.000000,0.000000,0.00000,0.012195,0.012195,0.012195,0.00,0.000000,0.012195,0.012195


In [214]:
toronto_grouped.shape

(38, 238)

### print each neighbourhood along with top five most popular venues

In [215]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2           Steakhouse  0.04
3  American Restaurant  0.04
4      Thai Restaurant  0.04


----Berczy Park----
            venue  freq
0     Coffee Shop  0.07
1    Cocktail Bar  0.05
2      Restaurant  0.03
3  Farmers Market  0.03
4            Café  0.03


----Brockton, Exhibition Place, Parkdale Village----
            venue  freq
0     Coffee Shop  0.14
1  Breakfast Spot  0.09
2            Café  0.09
3             Gym  0.05
4   Burrito Place  0.05


----Business reply mail Processing Centre969 Eastern----
                venue  freq
0         Yoga Studio  0.06
1       Auto Workshop  0.06
2  Light Rail Station  0.06
3       Garden Center  0.06
4              Garden  0.06


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
                venue  freq
0      Airport Lounge  0.14
1     Airport Service  0.14


## now put that data into a dataframe for furhter analysis

In [216]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [217]:
um_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,American Restaurant,Steakhouse,Thai Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Farmers Market,Bakery
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Café,Breakfast Spot,Gym / Fitness Center,Furniture / Home Store
3,Business reply mail Processing Centre969 Eastern,Yoga Studio,Auto Workshop,Garden Center,Garden,Light Rail Station
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Terminal,Airport Lounge,Harbor / Marina,Boat or Ferry
5,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Café,Pub,Indian Restaurant
6,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Japanese Restaurant,Bar
7,"Chinatown, Grange Park, Kensington Market",Café,Vegetarian / Vegan Restaurant,Chinese Restaurant,Bakery,Bar
8,Christie,Café,Grocery Store,Park,Restaurant,Nightclub
9,Church and Wellesley,Japanese Restaurant,Coffee Shop,Gay Bar,Sushi Restaurant,Restaurant


# Now let's cluster the neighbourhoods

### we'll use K-means algorithm and will start by clustering into 5 groups

In [226]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

### now create a merged data frame that includes
* info about each neighbourhood
* its cluster label
* the 5 top most common venues

In [227]:
toronto_merged = toronto_data

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,2,Coffee Shop,Bakery,Café,Park,Restaurant
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,2,Coffee Shop,Clothing Store,Café,Italian Restaurant,Japanese Restaurant
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,Coffee Shop,Restaurant,Hotel,Café,Park
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Coffee Shop,Pub,Pizza Place,Yoga Studio,Dim Sum Restaurant
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,2,Coffee Shop,Cocktail Bar,Seafood Restaurant,Farmers Market,Bakery


In [228]:
# create map to visualize the clusters
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine the clusters

In [229]:
# look at cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
24,"Parkdale, Roncesvalles",0,Gift Shop,Coffee Shop,Cuban Restaurant,Burger Joint,Dessert Shop


In [230]:
# look at cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
12,"Design Exchange, Toronto Dominion Centre",1,Coffee Shop,Hotel,Café,Gastropub,Deli / Bodega
28,"Moore Park, Summerhill East",1,Restaurant,Park,Tennis Court,Intersection,Electronics Store
36,Church and Wellesley,1,Japanese Restaurant,Coffee Shop,Gay Bar,Sushi Restaurant,Restaurant


In [231]:
# look at cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Harbourfront, Regent Park",2,Coffee Shop,Bakery,Café,Park,Restaurant
1,"Ryerson, Garden District",2,Coffee Shop,Clothing Store,Café,Italian Restaurant,Japanese Restaurant
2,St. James Town,2,Coffee Shop,Restaurant,Hotel,Café,Park
3,The Beaches,2,Coffee Shop,Pub,Pizza Place,Yoga Studio,Dim Sum Restaurant
4,Berczy Park,2,Coffee Shop,Cocktail Bar,Seafood Restaurant,Farmers Market,Bakery
5,Central Bay Street,2,Coffee Shop,Italian Restaurant,Café,Japanese Restaurant,Bar
6,Christie,2,Café,Grocery Store,Park,Restaurant,Nightclub
7,"Adelaide, King, Richmond",2,Coffee Shop,Café,American Restaurant,Steakhouse,Thai Restaurant
8,"Dovercourt Village, Dufferin",2,Bakery,Supermarket,Fast Food Restaurant,Music Venue,Liquor Store
9,"Harbourfront East, Toronto Islands, Union Station",2,Coffee Shop,Hotel,Pizza Place,Café,Italian Restaurant


In [232]:
# look at cluster 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
17,Lawrence Park,3,Dim Sum Restaurant,Bus Line,Park,Swim School,Falafel Restaurant
27,"Runnymede, Swansea",3,Coffee Shop,Café,Sushi Restaurant,Italian Restaurant,Restaurant


In [233]:
# look at cluster 5
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
22,North Toronto West,4,Clothing Store,Sporting Goods Shop,Coffee Shop,Yoga Studio,Chinese Restaurant
